# **Mentoring 1** - Credit Scoring and Analytics
Sekolah Data - Pacmann

In [3]:
# Load data manipulation package
import numpy as np
import pandas as pd


# Load data visualization package
import matplotlib.pyplot as plt
import seaborn as sns
from platformdirs import user_data_dir
from scipy.ndimage import percentile_filter

from tqdm import tqdm

In [4]:
PATH_CREDIT_RECORD = "credit_record.csv"
PATH_APPLICATION_RECORD = "application_record.csv"

---
**Mentoring 1 to assess your understanding of these topics**:
1. Data Preparation
2. Data Exploration

in scorecard development process.

## **Case**: Credit Card Performance
---
This is a fictive datasets from [Credit Card Approval Prediction](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction).

We have a performance table:
- `credit_record.csv` contains the credit performance of debtors.

We don't have the target or response variable in this table.

Therefore, we have to assign the response variable according to our definition of 'bad' credit performance.

For simplicity, you can access the dataset for mentoring from this folder: [Folder Dataset](https://drive.google.com/drive/folders/12Ea7Ence5mfK0UL7DJ2CgFrev7Uhrj3t?usp=sharing)

### **Data Preparation**:
---
Load the performance data from `credit_record.csv` file.

Variable explanation:
- `ID` : The account's unique ID.
- `MONTHS_BALANCE` : The month of the extracted data is the starting point, backwards.
  - `MONTHS_BALANCE = 0` is the current month,
  - `MONTHS_BALANCE = -1` is the previous month,
  - and so on.
- `STATUS` : The status of the account's balance.
  - `0` : 1-29 days past due
  - `1` : 30-59 days past due
  - `2` : 60-89 days past due
  - `3` : 90-119 days past due
  - `4` : 120-149 days past due
  - `5` : write-offs or more than 150 days past due
  - `C` : paid off that month
  - `X` : no loan for the month

In [5]:
# Import dataset from csv file
performance_data = pd.read_csv(PATH_CREDIT_RECORD)

# Table check
performance_data.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
# Check the data shape
performance_data.shape

(1048575, 3)

  - Performance data contains 3 variables from 1,048,575 records.
  - Let's check how many unique IDs we have.

In [7]:
# Check the total of unique IDs
len(set(performance_data['ID']))

45985

In [8]:
# Check data randomly
id_random = performance_data.sample(1, random_state=32)['ID']
id_random

317044    5029543
Name: ID, dtype: int64

In [9]:
# Check data with that random id
condition = performance_data['ID'].isin(id_random)
random_performance = performance_data[condition]
random_performance

,ID,MONTHS_BALANCE,STATUS
317035,5029543,-30,C
317036,5029543,-31,C
317037,5029543,-32,C
317038,5029543,-33,C
317039,5029543,-34,C
317040,5029543,-35,C
317041,5029543,-36,0
317042,5029543,-37,0
317043,5029543,-38,0
317044,5029543,-39,0


In [10]:
# Check data randomly
id_random2 = performance_data.sample(1, random_state=432)['ID']
id_random2

103547    5009469
Name: ID, dtype: int64

In [11]:
# Check data with that random id
condition2 = performance_data['ID'].isin(id_random2)
random_performance2 = performance_data[condition2]
random_performance2

,ID,MONTHS_BALANCE,STATUS
103547,5009469,0,0
103548,5009469,-1,1
103549,5009469,-2,0
103550,5009469,-3,0
103551,5009469,-4,0
103552,5009469,-5,0
103553,5009469,-6,X


### **Data Cleansing**:

In [12]:
# Check if status is valid
performance_data["STATUS"].value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

In [13]:
performance_data["MONTHS_BALANCE"].value_counts()

MONTHS_BALANCE
-1     34436
-2     34209
 0     33856
-3     33854
-4     33365
       ...  
-56     2109
-57     1649
-58     1255
-59      820
-60      415
Name: count, Length: 61, dtype: int64

In [14]:
only_x = performance_data.groupby("ID")["STATUS"].apply(lambda x: set(x) == {'X'})
only_x_ids = only_x[only_x].index.tolist()

performance_data = performance_data[~performance_data["ID"].isin(only_x_ids)]

In [15]:
first_month_users = performance_data.groupby("ID")["MONTHS_BALANCE"].max().sort_values(ascending=True)

In [16]:
first_month_users[first_month_users < 0].shape

(11210,)

In [17]:
performance_data["ID"].nunique()

41449

In [18]:
performance_data[performance_data["ID"] == 5035920]

,ID,MONTHS_BALANCE,STATUS
334514,5035920,-59,X
334515,5035920,-60,0


### **Question 1**:
What is the bad rate for a “bad” definition of 60+ days past due (DPD)?

---

**Expected Answer**:
```

print(f"The bad rate for a bad definition of 60+ DPD is about {bad_rate*100:.2f}%.")
```

In [19]:
# Get Bad-Rate User
# Formula to calculate bad-rate : (bad_user_ids/all_user_ids)
def has_60_plus_pass_due(x):
    statuses = {"2", "3", "4", "5"}
    x = set(x)
    return len(statuses.intersection(x)) > 0

bad_rate_users = performance_data.groupby("ID")["STATUS"].apply(lambda x: has_60_plus_pass_due(x.values))
bad_rate_users = bad_rate_users[bad_rate_users].index.tolist()
all_users = performance_data["ID"].nunique()

bad_rate = len(bad_rate_users) / all_users
print(f"The bad rate for a bad definition of 60+ DPD is about {bad_rate*100:.2f}%.")

The bad rate for a bad definition of 60+ DPD is about 1.61%.


### **Question 2**:

- How long do most people typically take to repay their loans?


[Hint: Use window repayment lenght and check the distribution before deciding the aggregation method]

**Expected Answer**:
```

print(f"Most people do repay their loan for {abs(int(loan_repayment_lenght))} month.")
```

In [43]:
open_month = performance_data.groupby("ID")["MONTHS_BALANCE"].max()
end_month = performance_data.groupby("ID")["MONTHS_BALANCE"].min()

repayment_data = pd.merge(max_month, min_month, on="ID")
repayment_data.index.name = "id"
repayment_data.columns = columns=["open_month", "end_month"]
repayment_data["window"] = repayment_data["open_month"] - repayment_data["end_month"]
repayment_data["window"].describe()

count    41449.000000
mean        21.848609
std         15.493808
min          0.000000
25%          9.000000
50%         18.000000
75%         33.000000
max         60.000000
Name: window, dtype: float64

In [45]:
print(f"Most people do repay their loan for 21.8 month.")

Most people do repay their loan for 21.8 month.


### **Question 3**:
What is the bad rate for a “bad” definition of 90+ days past due (DPD)?

---

**Expected Answer**:

```
print(f"The bad rate for a bad definition of 90+ DPD is about {bad_rate_90*100:.2f}%.")
```

In [46]:
# Get Bad-Rate User
# Formula to calculate bad-rate : (bad_user_ids/all_user_ids)
def has_60_plus_pass_due(x):
    statuses = {"3", "4", "5"}
    x = set(x)
    return len(statuses.intersection(x)) > 0

bad_rate_users = performance_data.groupby("ID")["STATUS"].apply(lambda x: has_60_plus_pass_due(x.values))
bad_rate_users = bad_rate_users[bad_rate_users].index.tolist()
all_users = performance_data["ID"].nunique()

bad_rate = len(bad_rate_users) / all_users
print(f"The bad rate for a bad definition of 60+ DPD is about {bad_rate*100:.2f}%.")

The bad rate for a bad definition of 60+ DPD is about 0.80%.


### **Question 4**:
Assume that the bad loan as an account whose loan status is writte-off (WO).
- Plot Cohort Table for Bad Rate
- Plot the most recent bad rate from each cohort (open month).
- Also define the performance window based on the bad rate cohort
---

In [246]:
performance_pivot2 = performance_pivot.copy()
performance_pivot2 = performance_pivot2[["id", "open_month", "end_month", "window"]]

performance_pivot2


MONTHS_BALANCE,id,open_month,end_month,window
0,5001711,-3,0,3
1,5001712,-18,0,18
2,5001713,-21,0,21
3,5001714,-14,0,14
4,5001715,-59,0,59
...,...,...,...,...
45980,5150482,-28,-11,17
45981,5150483,-17,0,17
45982,5150484,-12,0,12
45983,5150485,-1,0,1


### **Question 5**:
---

- Create a plot to compare cumulative bad rate between Accounts opened in 24, 36, 48, and 60 months ago
- Analyze the cumulative bad rate and provide insights that related to the sample window
- Based on the maturity curves you've created, if the sample window is four years ago (`open_month = -48`), what would the performance window be?
- Can we choose `open_month = -60` as the sample window? Why?

### **Question 6**:
---

- Create plots and compare the most recent bad rate using different  bad definition status:
    - `2` : 60-89 days past due
    - `3` : 90-119 days past due
    - `4` : 120-149 days past due
    - `5` : write-offs or more than 150 days past due

- Provide analysis of the most recent bad rate each of the status (Diagonal Cohort/The Most recent bad rate), compare each most recent bad rate according to all bad definition status

- Provide analysis, conclusion, and recommendation to sample window & sample window **using single cohort** in every bad status definition with these different open month: `[-36, -35, -25, -24]`

### **Question 7**:
---

- Read the `"application_record.csv"` from the dataset folder [Folder Dataset](https://drive.google.com/drive/folders/12Ea7Ence5mfK0UL7DJ2CgFrev7Uhrj3t?)
- Create bad_table for each bad definition status, create bad status as response variable in each bad definition status, and merge data with application data.
- Make it as a function `get_final_data(performance_data, application_data, bad_status_column, max_open=-36, min_open=-35)` with input open month default: `max_open = -36` and `min_open = -35` with output merged dataset
- Compare proportion between bad(1) and not bad(0) using `value_counts(normalize=True)` between all bad definition status from merged dataset
- Save all your data in csv format with these file name:
    - `dpd_60_data.csv`
    - `dpd_90_data.csv`
    - `dpd_120_data.csv`
    - `dpd_wo_data.csv`



Function example:
```
def get_final_data(performance_data, application_data, bad_status_column, max_open=-36, min_open=-35):
    """
    Function to get final data
    """

    # YOUR CODE HERE
    ...
    ...


    return data


# Example of usage
data = get_final_data(performance_data = performace,
                      application_data,
                      bad_status_column = "60+DPD",
                      max_open = -36,
                      min_open = -35)


```

Expected output:
- Cleaned application data from any ID duplicates
- 4 merged data given format and name above
- Cleaned from missing values
- Columns: `ID, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, CNT_CHILDREN, AMT_INCOME_TOTAL, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE,  NAME_FAMILY_STATUS,	NAME_HOUSING_TYPE, DAYS_BIRTH, DAYS_EMPLOYED, FLAG_MOBIL, FLAG_WORK_PHONE, FLAG_PHONE, FLAG_EMAIL, OCCUPATION_TYPE, CNT_FAM_MEMBERS, open_month, BAD.`